# Package Usage Example

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
from tqdm import tqdm
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn import preprocessing
from sklearn.model_selection import cross_val_score
from sklearn.utils import resample

import torch
from torch.utils.data import Dataset,DataLoader
from torch import optim
import torch.nn as nn
import torch.nn.functional as F

from Multi-Layer-Kernel-Machine.Mydataset import mydataset
from Multi-Layer-Kernel-Machine.RandomFeature import RandomFourierFeature
from Multi-Layer-Kernel-Machine.Fitting import MultilayerFitting
from Multi-Layer-Kernel-Machine.Structure import Net,ResNet,KernelNet,ResKernelNet
from Multi-Layer-Kernel-Machine.DataSplitting import DataSplitting
from Multi-Layer-Kernel-Machine.GenerateSplit import GenerateSplit

In [ ]:
##### example
df=pd.read_csv('YearPredictionMSD.txt',header=None,sep = ',')
#smaller dataset
train=df.iloc[300000:300700] 
test=df.iloc[500000:500300] 
train_y = train[0]
test_y = test[0] #response:year
train_x = train
test_x = test
del train_x[0]
del test_x[0]
total_x=pd.concat([train_x,test_x])
x=preprocessing.StandardScaler().fit(total_x).transform(total_x) #normalize
x=pd.DataFrame(x)
train_x=x.iloc[:700]
test_x=x.iloc[700:]

train_x.reset_index(drop=True, inplace=True) 
test_x.reset_index(drop=True, inplace=True) 
train_y.reset_index(drop=True, inplace=True) 
test_y.reset_index(drop=True, inplace=True) 

train_x ##display the data

In [ ]:
##### data
nntrain_x = torch.from_numpy(train_x.to_numpy()).float()
nntrain_y = torch.squeeze(torch.from_numpy(train_y.to_numpy()).float()) 
nntest_x= torch.from_numpy(test_x.to_numpy()).float()
nntest_y = torch.squeeze(torch.from_numpy(test_y.to_numpy()).float())
    
    
batch=100
train_loader = DataLoader(mydataset(nntrain_x, nntrain_y),batch_size=batch, shuffle=True)
test_loader = DataLoader(mydataset(nntest_x, nntest_y),batch_size=batch, shuffle=False)

## Multilayer Structure

In [ ]:
##### DNN example
def init_weights(m):
    if type(m) == nn.Conv2d:
        torch.nn.init.normal_(m.weight,mean=0,std=0.5)
    if type(m) == nn.Linear:
        torch.nn.init.uniform_(m.weight,a=0,b=0.1)
        m.bias.data.fill_(0.01)  
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
net = Net([90,32,8,1],device) 
print(net)
torch.manual_seed(0)
net.apply(init_weights)
criterion=nn.MSELoss() 
optimizer=optim.SGD(net.parameters(),lr=1e-6,momentum=0.9,weight_decay=1e-2) #optim.Adam(...)
mlmodel=MultilayerFitting(net,device,train_loader, test_loader, 2000, criterion, optimizer,100,100)
dnn_trainloss,dnn_testloss,dnn_bootbase=mlmodel.fitting(train_x,train_y,test_x,test_y, batch)
#mlmodel.Bootstrap(40,dnn_bootbase,train_x,train_y,test_x,test_y, batch,init_weights)

In [ ]:
##### ResNet example
def init_weights(m):
    if type(m) == nn.Conv2d:
        torch.nn.init.normal_(m.weight,mean=0,std=0.5)
    if type(m) == nn.Linear:
        torch.nn.init.uniform_(m.weight,a=0,b=0.1)
        m.bias.data.fill_(0.01)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
net = ResNet([90,32,8,1],device)
print(net)
torch.manual_seed(0)
net.apply(init_weights)
criterion=nn.MSELoss() 
optimizer=optim.SGD(net.parameters(),lr=1e-7,momentum=0.9,weight_decay=1e-2)
mlmodel=MultilayerFitting(net,device,train_loader, test_loader, 4000, criterion, optimizer,100,100)
res_trainloss,res_testloss,res_bootbase=mlmodel.fitting(train_x,train_y,test_x,test_y, batch)
#mlmodel.Bootstrap(40,res_bootbase,train_x,train_y,test_x,test_y, batch,init_weights)

In [ ]:
##### MLK example
def init_weights(m):
    if type(m) == nn.Conv2d:
        torch.nn.init.normal_(m.weight,mean=0,std=0.5)
    if type(m) == nn.Linear:
        torch.nn.init.uniform_(m.weight,a=0,b=1)
        m.bias.data.fill_(0.01)
        
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
net = KernelNet([90,32,8,1],["C","G"],[0.01,0.1],device)
print(net)
torch.manual_seed(1)
net.apply(init_weights)
criterion=nn.MSELoss() 
optimizer=optim.SGD(net.parameters(),lr=1e-3,momentum=0.9,weight_decay=1e-4) #optim.Adam(...)
mlmodel=MultilayerFitting(net,device,train_loader, test_loader, 2000, criterion, optimizer,100,100)
kernelnn_trainloss,kernelnn_testloss,kernelnn_bootbase=mlmodel.fitting(train_x,train_y,test_x,test_y, batch)
#mlmodel.Bootstrap(40,kernelnn_bootbase,train_x,train_y,test_x,test_y, batch,init_weights)

In [ ]:
### RK example
def init_weights(m):
    if type(m) == nn.Conv2d:
        torch.nn.init.normal_(m.weight,mean=0,std=0.5)
    if type(m) == nn.Linear:
        torch.nn.init.uniform_(m.weight,a=0,b=1)
        m.bias.data.fill_(0.01)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
net = ResKernelNet([90,32,8,1],["C","G"],[0.01,0.1],device)
print(net)
torch.manual_seed(1)
net.apply(init_weights)
criterion=nn.MSELoss() 
criterion=nn.MSELoss() 
optimizer=optim.SGD(net.parameters(),lr=1e-6,momentum=0.9,weight_decay=1e-2) #optim.Adam(...)
mlmodel=MultilayerFitting(net,device,train_loader, test_loader, 2000, criterion, optimizer,100,100)
reskernel_trainloss,reskernel_testloss,reskernel_bootbase=mlmodel.fitting(train_x,train_y,test_x,test_y, batch)
#mlmodel.Bootstrap(40,reskernel_bootbase,train_x,train_y,test_x,test_y, batch,init_weights)

In [ ]:
### single layer example
model =Ridge(alpha=1)
score=[]
trainmse=[]
testmse=[]
gaset=np.linspace(0,0.3,10)
for ga in gaset:
    rff=RandomFourierFeature(90,100,kernel='G',gamma=ga,device="cpu")
    train_xtransformed=rff.transform(nntrain_x)
    test_xtransformed=rff.transform(nntest_x)
    model.fit(train_xtransformed, nntrain_y)
    y_pretest=model.predict(test_xtransformed)
    y_pretrain=model.predict(train_xtransformed)
    score.append(cross_val_score(model,train_xtransformed,train_y,cv=5).mean())   ##cross-validation
    trainmse.append(mean_squared_error(y_pretrain,train_y))
    testmse.append(mean_squared_error(y_pretest,test_y))
ind=score.index(max(score))
print("cv-score,index,gamma,trainmse,testmse",max(score),ind,gaset[ind],trainmse[ind],testmse[ind])

estimation

In [ ]:
##### MLK interval example
def init_weights(m):
    if type(m) == nn.Conv2d:
        torch.nn.init.normal_(m.weight,mean=0,std=0.5)
    if type(m) == nn.Linear:
        torch.nn.init.uniform_(m.weight,a=0,b=1)
        m.bias.data.fill_(0.01)
        
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
net = KernelNet([90,32,8,1],["C","G"],[0.01,0.1],device)
print(net)
torch.manual_seed(1)
net.apply(init_weights)
criterion=nn.MSELoss() 
optimizer=optim.SGD(net.parameters(),lr=1e-3,momentum=0.9,weight_decay=1e-4) #optim.Adam(...)
mlmodel=MultilayerFitting(net,device,train_loader, test_loader, 2000, criterion, optimizer,100,100,printchoice=False)
kernelnn_trainloss,kernelnn_testloss,kernelnn_bootbase=mlmodel.fitting(train_x,train_y,test_x,test_y, batch)

mlmodel.GradientBand(kernelnn_trainloss,train_x,train_y,test_x,test_y, batch)
mlmodel.HomoConformalBand(train_x,train_y,test_x,test_y, batch)
mlmodel.HeteConformalBand(kernelnn_trainloss,train_x,train_y,test_x,test_y, batch)

comparison

In [ ]:
def plot_choose(fig,start,data):
    fig.plot(np.arange(start,len(data)),data[start:])

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(10, 4))
plot_choose(ax[0],100,dnn_testloss)
plot_choose(ax[0],100,kernelnn_testloss)
ax[0].legend(["DNN","Multi_Kernel"])
ax[0].set_xlabel("epoch")
ax[0].set_ylabel("Test MSE")
ax[0].set_title("Test MSE with different structure")

plot_choose(ax[1],100,res_testloss)
plot_choose(ax[1],100,reskernel_testloss)
ax[1].legend(["Resnet","Residual_Multi_Kernel"])
ax[1].set_xlabel("epoch")
ax[1].set_ylabel("Test MSE")
ax[1].set_title("Test MSE with different structure")

plt.show()

## data splitting

In [ ]:
def init_weights(m):
    if type(m) == nn.Conv2d:
        torch.nn.init.normal_(m.weight,mean=0,std=0.5)
    if type(m) == nn.Linear:
        torch.nn.init.uniform_(m.weight,a=0,b=1)
        m.bias.data.fill_(0.01)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
net = KernelNet([90,32,8,1],["C","G"],[0.01,0.1],device)
train_loaderset,netset,optimizerset=GenerateSplit(2,device,net,8e-4,0.9,1e-4,
                                                                       train_x,train_y, batch,init_weights)
criterion=nn.MSELoss()  
split=2  
splkermodel=DataSplitting(split,netset,device,train_loaderset, test_loader, 2000, criterion, optimizerset,100,100,printchoice=True)
splker_trainloss,splker_testloss,splker_prediction=splkermodel.fitting(train_x,train_y,test_x,test_y, batch)

In [ ]:
splkermodel.GradientBand(splker_trainloss,train_x,train_y,test_x,test_y, batch)

In [ ]:
def init_weights(m):
    if type(m) == nn.Conv2d:
        torch.nn.init.normal_(m.weight,mean=0,std=0.5)
    if type(m) == nn.Linear:
        torch.nn.init.uniform_(m.weight,a=0,b=1)
        m.bias.data.fill_(0.01)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
net = ResKernelNet([90,32,8,1],["C","G"],[0.01,0.1],device)
train_loaderset,netset,optimizerset=GenerateSplit(2,device,net,1e-6,0.9,1e-4,
                                                                       train_x,train_y, batch,init_weights)
criterion=nn.MSELoss()    
split=2 
splresmodel=DataSplitting(split,netset,device,train_loaderset, test_loader, 2000, criterion, optimizerset,100,100,printchoice=True)
splres_trainloss,splres_testloss,splres_prediction=splresmodel.fitting(train_x,train_y,test_x,test_y, batch)

In [ ]:
splresmodel.GradientBand(splres_trainloss,train_x,train_y,test_x,test_y, batch)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(12, 4))
plot_choose(ax[0],200,dnn_testloss)
plot_choose(ax[0],200,res_testloss)
plot_choose(ax[0],200,kernelnn_testloss)
plot_choose(ax[0],200,reskernel_testloss)
ax[0].legend(["DNN","ResNet","MLKM","RKM"],fontsize="x-large")
ax[0].set_xlabel("epoch",fontsize="xx-large")
ax[0].set_ylabel("Test MSE",fontsize="xx-large")

plot_choose(ax[1],200,dnn_trainloss)
plot_choose(ax[1],200,res_trainloss)
plot_choose(ax[1],200,kernelnn_trainloss)
plot_choose(ax[1],200,reskernel_trainloss)
ax[1].legend(["DNN","ResNet","MLKM","RKM"],fontsize="x-large")
ax[1].set_xlabel("epoch",fontsize="xx-large")
ax[1].set_ylabel("Train MSE",fontsize="xx-large")
plt.show()